In [11]:
from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
from torch.utils.data.sampler import SubsetRandomSampler, Sampler

from torchvision import transforms

In [12]:
# First, lets load the dataset
tfs = transforms.Compose([
    transforms.ColorJitter(hue=.20, saturation=.20),
    transforms.RandomRotation(10),
    transforms.RandomAdjustSharpness(3, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43,0.44,0.47],
                       std=[0.20,0.20,0.20])
])
data_train = dset.SVHN('./data/',
                       transform=tfs
                      )
data_test = dset.SVHN('./data/', split='test', transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.43,0.44,0.47],
                                               std=[0.20,0.20,0.20])
                       ]))

In [13]:
batch_size = 64

data_size = data_train.data.shape[0]
indices = list(range(data_size))
np.random.shuffle(indices)

train_sampler = SubsetRandomSampler(indices)
train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                           sampler=train_sampler)

In [14]:
def train_model(model, train_loader, val_loader, loss, optimizer, scheduler, num_epochs):
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        scheduler.step()
        model.train() # Enter train mode

        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            prediction = model(x)
            loss_value = loss(prediction, y)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()

            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y)
            total_samples += y.shape[0]

            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        #val_accuracy = compute_accuracy(model, val_loader)
        val_accuracy = 0

        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        print("epoch %d Average loss: %f, Train accuracy: %f, val accuracy: %f" %
                  (epoch+1, ave_loss, train_accuracy, val_accuracy))
        #if train_accuracy < 0.15:
        #    return loss_history, train_history, val_history

    return loss_history, train_history, val_history

def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader

    Returns: accuracy as a float value between 0 and 1
    """
    model.eval()
    correct_samples = 0
    total_samples = 0
    for i_step, (x, y) in enumerate(loader):
        # x_gpu = x.to(device)
        # y_gpu = y.to(device)
        prediction = model(x)
        _, indices = torch.max(prediction, 1)
        correct_samples += torch.sum(indices == y)
        total_samples += y.shape[0]

    train_accuracy = float(correct_samples) / total_samples
    return train_accuracy

#loss_history, train_history, val_history = train_model(nn_model, train_loader, val_loader, loss, optimizer, 5)

In [15]:
# We'll use a special helper module to shape it into a flat tensor
class Flattener(nn.Module):
    def forward(self, x):
        batch_size, *_ = x.shape
        return x.view(batch_size, -1)

In [16]:
epoch_num = 30
loss = nn.CrossEntropyLoss().type(torch.FloatTensor)

my_model = nn.Sequential(
  nn.Conv2d(3, 6, 5, padding=0),
  nn.ReLU(inplace=True),
  nn.BatchNorm2d(6),
  nn.MaxPool2d(2),
  nn.Conv2d(6, 16, 5, padding=0),
  nn.ReLU(inplace=True),
  nn.BatchNorm2d(16),
  nn.MaxPool2d(2),
  Flattener(),
  nn.Linear(16*5*5, 120),
  nn.ReLU(inplace=True),
  nn.BatchNorm1d(120),
  nn.Linear(120, 84),
  nn.ReLU(inplace=True),
  nn.BatchNorm1d(84),
  nn.Linear(84, 10)
  )

lr = 0.0061
reg = 3.4e-5
gamma = 0.5
step_size = 4

optimizer = optim.Adam(my_model.parameters(), lr=lr, weight_decay=reg)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
loss_history, train_history, val_history = train_model(my_model, train_loader, 0, loss, optimizer, scheduler, epoch_num)

C:\Users\lavre\Desktop\dlcourse_ai\venv\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


epoch 1 Average loss: 0.705662, Train accuracy: 0.775312, val accuracy: 0.000000
epoch 2 Average loss: 0.468487, Train accuracy: 0.855399, val accuracy: 0.000000
epoch 3 Average loss: 0.429603, Train accuracy: 0.868695, val accuracy: 0.000000
epoch 4 Average loss: 0.360432, Train accuracy: 0.889526, val accuracy: 0.000000
epoch 5 Average loss: 0.347881, Train accuracy: 0.892993, val accuracy: 0.000000
epoch 6 Average loss: 0.334144, Train accuracy: 0.897607, val accuracy: 0.000000
epoch 7 Average loss: 0.325838, Train accuracy: 0.900788, val accuracy: 0.000000
epoch 8 Average loss: 0.283851, Train accuracy: 0.912882, val accuracy: 0.000000
epoch 9 Average loss: 0.274749, Train accuracy: 0.914766, val accuracy: 0.000000
epoch 10 Average loss: 0.267385, Train accuracy: 0.917810, val accuracy: 0.000000
epoch 11 Average loss: 0.260996, Train accuracy: 0.918847, val accuracy: 0.000000


KeyboardInterrupt: 

In [19]:
data_train = dset.SVHN('./data/', transform = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.43,0.44,0.47],
                                               std=[0.20,0.20,0.20])]))
train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                           sampler=train_sampler)
loss_history, train_history, val_history = train_model(my_model, train_loader, 0, loss, optimizer, scheduler, 5)

epoch 1 Average loss: 0.182880, Train accuracy: 0.944838, val accuracy: 0.000000
epoch 2 Average loss: 0.168451, Train accuracy: 0.949411, val accuracy: 0.000000
epoch 3 Average loss: 0.145663, Train accuracy: 0.956414, val accuracy: 0.000000
epoch 4 Average loss: 0.138396, Train accuracy: 0.958489, val accuracy: 0.000000
epoch 5 Average loss: 0.129452, Train accuracy: 0.961696, val accuracy: 0.000000


In [20]:
test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)
test_accuracy = compute_accuracy(my_model, test_loader)
print("Test accuracy: %2.4f" % test_accuracy)

Test accuracy: 0.9132
